In [ ]:
# The code below avoids libraries as much as possible to build understanding 
# of what methods do and how neural networks work under the hood

In [24]:
import torchvision.datasets as datasets
from PIL.ImageShow import show
from PIL.Image import Image
from typing import List, Union
import random

In [21]:
# Constants
ALPHA = 0.01
NORMALISATION = 255
BATCH_SIZE = 16

In [3]:
# Load MNIST data set
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=None)

# Get first image data and target and display image
train_image_zero, train_target_zero = train_data[0]
image_zero_pixels = train_image_zero.getdata()
number_pixels = len(image_zero_pixels)

train_image_zero.show()
print(f"The first label is a {train_target_zero}.")
print(f"There are {number_pixels} pixels in the first image.")

The first label is a 5.
There are 784 pixels in the first image.


In [52]:
# Functions to train neural net 
def w_sum(inputs: List[List[Union[float, int]]], weights: List[Union[float, int]]) -> List[Union[float, int]]:
    m = len(inputs)
    n = len(weights)
    if len(inputs[0]) != n:
        raise ValueError("Vectors need to be of same len")
    
    outputs = [0] * m
    
    for i in range(m):
        for j in range(n):
            outputs[i] += inputs[i][j] * weights[j]
    return outputs

def calculate_deltas(predicted_labels: List[Union[float, int]], labels: List[Union[float, int]]) -> List[Union[float, int]]:
    n = len(predicted_labels)

    if len(labels) != n:
        raise ValueError("Vectors need to be of same len")

    deltas = [0] * n

    for i in range(n):
        deltas[i] = predicted_labels[i] - labels[i]

    return deltas

def calculate_errors(deltas: List[float]) -> List[float]:
    n = len(deltas)
    errors = [0] * n

    for i in range(n):
        errors[i] = deltas[i] ** 2

    return errors

def neural_network(inputs, weights, targets):
    preds = w_sum(inputs, weights)
    deltas = calculate_deltas(preds, targets)
    errors = calculate_errors(deltas)
    
    return preds, deltas, errors
    
def calculate_weight_deltas(inputs: List[Union[float, int]], deltas: List[float]) -> List[List[float]]:
    m = len(inputs)
    n = len(inputs[0])

    #     # Debugging statements
    # print(f"inputs shape: ({m}, {n})")
    # print(f"deltas length: {len(deltas)}")
    # print(f"deltas content: {deltas}")

    # if len(deltas) != n:
    #     raise ValueError("Length of deltas must match the number of features (columns) in inputs.")

    weight_deltas = [[0] * n for _ in range(m)]

    for i in range(m):
        for j in range(n):
            weight_deltas[i][j] = inputs[i][j] * deltas[i]
    
    return weight_deltas

def back_propagation(weights: List[Union[float, int]], weight_deltas: List[List[float]]) -> None:
    length = len(weights)
    avg_deltas = [0] * length   

    for i in range(length):
        for delta in weight_deltas:
            avg_deltas[i] += delta[i] / len(weight_deltas)

    for i in range(length):
        weights[i] -= ALPHA * avg_deltas[i]

def forward_propagation(inputs, weights, target):
    preds, deltas, errors = neural_network(inputs, weights, target)
    weight_deltas = calculate_weight_deltas(inputs, deltas)                                       
    back_propagation(weights, weight_deltas)
    return preds, errors

def train(data):
    weights = [random.uniform(-0.5, 0.5) for _ in range(number_pixels)]

    for i in range(0, len(train_data), BATCH_SIZE):
        batch = train_data[i: i + BATCH_SIZE]
        input_data = [[pixel / NORMALISATION for pixel in list(x[0].getdata())] for x in batch]
        targets = [x[1] for x in batch] 

        pred, error = forward_propagation(input_data, weights, targets)

        if i % (100 * BATCH_SIZE) == 0: 
            print(f"Batch {(i / BATCH_SIZE) + 1}: Last batch pred: {pred[-1]}, Last batch target: {targets[-1]}, Avg error: {sum(error) / len(error)}")
    
    return weights

In [48]:
# Set up test functions
def test_w_sum():
    assert w_sum([[1, 2, 3], [4, 5, 6]], [1, 2, 3]) == [14, 32]

def test_calculate_deltas():
    assert calculate_deltas([1, 2, 3], [5, 5, 5]) == [-4, -3, -2]

def test_calculate_errors():
    assert calculate_errors([-4, -3, -2]) == [16, 9, 4]

def test_calculate_weight_deltas():
    assert calculate_weight_deltas([[1, 2, 3], [4, 5, 6]], [-4, -3, -2]) == [[-4, -6, -6], [-16, -15, -12]]

def test_back_propagation():
    weights = [1, 2, 3]
    weight_deltas = [[-4, -6, -6], [-16, -15, -12]]
    back_propagation(weights, weight_deltas)
    assert weights == [1.1, 2.105, 3.09]

# Run tests
test_w_sum()
test_calculate_errors()
test_calculate_deltas()
test_calculate_weight_deltas()
test_back_propagation()

In [53]:
train_data = [(image, label) for image, label in train_data]
weights = train(train_data)


Batch 1.0: Last batch pred: -1.0613820536515812, Last batch target: 7, Avg error: 30.865630027126244
Batch 101.0: Last batch pred: 2.9815932818995816, Last batch target: 5, Avg error: 4.265678021389187
Batch 201.0: Last batch pred: 0.8980639537703442, Last batch target: 3, Avg error: 6.693346798378457
Batch 301.0: Last batch pred: 0.028064552322429114, Last batch target: 1, Avg error: 5.780141476148991
Batch 401.0: Last batch pred: 1.8464604327098315, Last batch target: 3, Avg error: 2.2268141224850995
Batch 501.0: Last batch pred: 8.13119043336007, Last batch target: 9, Avg error: 4.3254348371116365
Batch 601.0: Last batch pred: 1.087057073033319, Last batch target: 1, Avg error: 4.126560449848958
Batch 701.0: Last batch pred: 7.298928494298924, Last batch target: 9, Avg error: 5.929020699520281
Batch 801.0: Last batch pred: -0.477930246857102, Last batch target: 2, Avg error: 1.5688087747529753
Batch 901.0: Last batch pred: 7.385143398519008, Last batch target: 4, Avg error: 7.296864